In [1]:
import torch.nn.functional as F

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True, cache_dir="/home/maxim/models")
# model_2 = AutoModelForCausalLM.from_pretrained(
#     "deepseek-ai/deepseek-coder-1.3b-base", 
#     trust_remote_code=True, 
#     cache_dir="/home/maxim/models",
#     attn_implementation="eager",
#     # torch_dtype=torch.float16,
# ).cuda()
# model_2 = model_2.half()
# print(model)
# print(model.config)

/home/maxim/miniconda3/envs/dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
checkpoint = "bigcode/tiny_starcoder_py"
device = "cuda" # for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint, cache_dir="/home/maxim/models")
# model = AutoModelForCausalLM.from_pretrained(checkpoint, cache_dir="/home/maxim/models")

In [4]:
model.config

GPTBigCodeConfig {
  "_name_or_path": "bigcode/tiny_starcoder_py",
  "activation_function": "gelu_pytorch_tanh",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": 3072,
  "n_layer": 20,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "resid_pdrop": 0.1,
  "scale_attention_softmax_in_fp32": true,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.41.1",
  "use_cache": true,
  "validate_runner_input": tr

In [3]:
from generator.models.Bitnet.modeling_bitnet import BitnetForCausalLM
from generator.models.Bitnet.configuration_bitnet import BitnetAttentionConfig, BitnetConfig, BitnetFFNConfig
from generator.models.Llama.modeling_llama import LlamaForCausalLM
from generator.models.GPTBigcode.modeling_gpt_bigcode import GPTBigCodeForCausalLM

%load_ext autoreload
%autoreload 2

In [4]:
# model_parent_config = AutoConfig.from_pretrained(
#     "deepseek-ai/deepseek-coder-1.3b-base", 
#     trust_remote_code=True, 
#     cache_dir="/home/maxim/models",
#     attn_implementation="eager",
# )

model_parent_config = AutoConfig.from_pretrained(
    "bigcode/tiny_starcoder_py", 
    trust_remote_code=True, 
    cache_dir="/home/maxim/models",
    attn_implementation="eager",
)

/home/maxim/miniconda3/envs/dl/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
model_parent_config

GPTBigCodeConfig {
  "_name_or_path": "bigcode/tiny_starcoder_py",
  "activation_function": "gelu_pytorch_tanh",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": 3072,
  "n_layer": 20,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "resid_pdrop": 0.1,
  "scale_attention_softmax_in_fp32": true,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.41.1",
  "use_cache": true,
  "validate_runner_input": tr

In [5]:
model_path = '/home/maxim/models/models--deepseek-ai--deepseek-coder-1.3b-base/snapshots/c919139c3a9b4070729c8b2cca4847ab29ca8d94'
model_path = '/home/maxim/models/models--bigcode--tiny_starcoder_py/snapshots/8547527bef0bc927268c1653cce6948c5c242dd1'
# parent_model = LlamaForCausalLM.from_pretrained(
#     config=model_parent_config,
#     pretrained_model_name_or_path=model_path,
# ).cuda()

parent_model = GPTBigCodeForCausalLM.from_pretrained(
    config=model_parent_config,
    pretrained_model_name_or_path=model_path,
).cuda()

In [9]:
parent_model.config

GPTBigCodeConfig {
  "_name_or_path": "/home/maxim/models/models--bigcode--tiny_starcoder_py/snapshots/8547527bef0bc927268c1653cce6948c5c242dd1",
  "activation_function": "gelu_pytorch_tanh",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": 3072,
  "n_layer": 20,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "resid_pdrop": 0.1,
  "scale_attention_softmax_in_fp32": true,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "tran

In [8]:
len(tokenizer)

49152

In [9]:
attention_config = BitnetAttentionConfig(
    kv_n_heads=12,
)

ffn_config = BitnetFFNConfig(
    ffn_hidden_size=768,
)

model_config = BitnetConfig(
    attn_implementation="eager",
    d_model=384,
    n_heads=12,
    n_layers=20,
    max_seq_len=256,
    use_last_bit_linear=False,
    vocab_size=len(tokenizer),
    attn_config=attention_config,
    ffn_config=ffn_config,
)

model = BitnetForCausalLM(model_config).to("cuda")
# model = BitnetForCausalLM(model_config)

In [4]:
model

BitnetForCausalLM(
  (model): BitnetModel(
    (embed_tokens): Embedding(32022, 512)
    (layers): ModuleList(
      (0-23): 24 x BitnetDecoderLayer(
        (self_attn): BitnetAttention(
          (q_proj): BitLinear(
            in_features=512, out_features=512, bias=False
            (norm): RMSNorm()
          )
          (k_proj): BitLinear(
            in_features=512, out_features=512, bias=False
            (norm): RMSNorm()
          )
          (v_proj): BitLinear(
            in_features=512, out_features=512, bias=False
            (norm): RMSNorm()
          )
          (o_proj): BitLinear(
            in_features=512, out_features=512, bias=False
            (norm): RMSNorm()
          )
          (rotary_emb): BitnetRotaryEmbedding()
        )
        (mlp): BitnetMLP(
          (gate_proj): BitLinear(
            in_features=512, out_features=768, bias=False
            (norm): RMSNorm()
          )
          (up_proj): BitLinear(
            in_features=512, out_featu

In [4]:
model_2.model.embed_tokens

Embedding(32256, 2048)

In [10]:
from torch import nn

In [11]:
class ToSmallEmb(nn.Module):
    def __init__(self, emb, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim, bias=False)
        self.emb = emb

    def forward(self, x):
        # print(x)
        # print(self.emb(x))
        with torch.no_grad():
            emb = self.emb(x)
        return self.linear(emb)

In [27]:
parent_model

GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49152, 768)
    (wpe): Embedding(8192, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-19): 20 x GPTBigCodeBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeAttention(
          (c_attn): Linear(in_features=768, out_features=896, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          (act): PytorchGELUTanh()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elemen

In [12]:
# to_small = ToSmallEmb(parent_model.model.embed_tokens, 2048, 512).to("cuda").to(torch.float32)
# to_small = ToSmallEmb(parent_model.model.embed_tokens, 2048, 512).to("cuda")
to_small = ToSmallEmb(parent_model.transformer.wte, 768, 384).to("cuda")

In [13]:
model.model.embed_tokens = to_small

In [14]:
model_size = sum(t.numel() for t in model.parameters())

In [15]:
model_size

86471040

In [16]:
print(f"bitnet size: {model_size/1000**2:.1f}M parameters")

bitnet size: 86.5M parameters


In [16]:
?model.__call__

Signature: model.__call__(*args, **kwargs)
Docstring: <no docstring>
File:      ~/miniconda3/envs/dl/lib/python3.11/site-packages/torch/nn/modules/module.py
Type:      method

In [16]:
tokenizer.eos_token

'<|endoftext|>'

In [18]:
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

0

In [19]:
inputs_ids = tokenizer(["hello my friend, How are", "hi"], return_tensors="pt", padding=True)

In [19]:
inputs_ids

{'input_ids': tensor([[ 7656,  1672, 11970,    30,  4971,   884],
        [ 4980,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0]])}

In [21]:
inputs_ids

{'input_ids': tensor([[ 7656,  1672, 11970,    30,  4971,   884],
        [ 4980,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0]])}

In [21]:
inputs_ids = inputs_ids['input_ids'].to("cuda")

In [22]:
output_2 = parent_model(inputs_ids,  return_attentions_before_softmax=True, output_attentions=True)

In [24]:
output_2.keys()

odict_keys(['logits', 'past_key_values', 'attentions'])

In [24]:
output_2['attentions'][0].shape

torch.Size([2, 12, 6, 6])

In [30]:
p_log = F.log_softmax(output_2['attentions'][0], dim=-1)

In [31]:
p_log

tensor([[[[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38],
          [-1.1131e-01, -2.2506e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38],
          [-2.5084e-01, -2.1282e+00, -2.2749e+00, -3.4028e+38, -3.4028e+38,
           -3.4028e+38],
          [-5.6460e-01, -1.8871e+00, -1.4044e+00, -3.3703e+00, -3.4028e+38,
           -3.4028e+38],
          [-6.3499e-01, -2.0290e+00, -2.0589e+00, -1.8960e+00, -2.7996e+00,
           -3.4028e+38],
          [-1.0791e+00, -1.8591e+00, -2.4366e+00, -2.0741e+00, -2.5895e+00,
           -1.5319e+00]],

         [[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38],
          [-1.4791e-01, -1.9842e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
           -3.4028e+38],
          [-2.1724e-01, -2.0915e+00, -2.6344e+00, -3.4028e+38, -3.4028e+38,
           -3.4028e+38],
          [-5.3942e-01, -2.5632e+00, -3.0188e+00, -1.2345e+00, -3.4028e+38,
           -3

In [29]:
p_log_real = F.log_softmax(output['attentions'][0][1], dim=-1)

In [33]:
p = F.softmax(output['attentions'][0][1], dim=-1)

In [41]:
(p * (p_log - p_log_real)).sum(axis=-1).mean(axis=(-1, -2))

tensor([-0.3213, -0.2210], device='cuda:0', grad_fn=<MeanBackward1>)

In [13]:
len(output_2['attentions'])

24

In [15]:
b = F.log_softmax(output_2['attentions'][0][:, 1], dim=-1)

In [17]:
b.shape

torch.Size([2, 7, 7])

In [23]:
# output = model(inputs_ids['input_ids'].to("cuda"), output_attentions=True)
output = model(inputs_ids, output_attentions=True, return_attentions_before_softmax=True)

In [9]:
output.logits.shape

torch.Size([1, 7, 32022])

In [11]:
a = torch.nn.functional.softmax(output.logits, dim=-1)

In [13]:
a.sum(axis=-1)

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],
       device='cuda:0', grad_fn=<SumBackward1>)

In [10]:
output_2 = model_2(inputs_ids,  output_attentions=True)

In [11]:
output.keys()

odict_keys(['logits', 'past_key_values', 'attentions'])

In [37]:
len(output.attentions)

24

In [68]:
output['attentions'][0][0][0, 1]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4901, 0.5099, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3610, 0.3349, 0.3041, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2849, 0.2415, 0.2467, 0.2269, 0.0000, 0.0000, 0.0000],
        [0.1930, 0.1915, 0.1871, 0.2247, 0.2038, 0.0000, 0.0000],
        [0.1806, 0.1618, 0.1765, 0.1523, 0.1649, 0.1640, 0.0000],
        [0.1463, 0.1421, 0.1339, 0.1563, 0.1445, 0.1388, 0.1381]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [44]:
a = F.log_softmax(output['attentions'][0][0, 1], dim=-1)

In [45]:
a_p = F.softmax(output['attentions'][0][0, 1], dim=-1)

In [47]:
a_p

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5367, 0.4633, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3252, 0.3354, 0.3395, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2541, 0.2816, 0.2119, 0.2524, 0.0000, 0.0000, 0.0000],
        [0.1889, 0.2360, 0.2040, 0.1908, 0.1804, 0.0000, 0.0000],
        [0.1747, 0.1630, 0.1596, 0.1760, 0.1813, 0.1455, 0.0000],
        [0.1313, 0.1751, 0.1311, 0.1388, 0.1336, 0.1461, 0.1440]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [49]:
b

tensor([[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
         -3.4028e+38, -3.4028e+38],
        [-1.6772e-01, -1.8682e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
         -3.4028e+38, -3.4028e+38],
        [-3.9324e-01, -1.8360e+00, -1.7978e+00, -3.4028e+38, -3.4028e+38,
         -3.4028e+38, -3.4028e+38],
        [-8.2617e-01, -1.4385e+00, -1.6120e+00, -2.0754e+00, -3.4028e+38,
         -3.4028e+38, -3.4028e+38],
        [-5.2994e-01, -2.4086e+00, -2.4951e+00, -2.8222e+00, -1.7178e+00,
         -3.4028e+38, -3.4028e+38],
        [-1.1486e+00, -1.9705e+00, -1.7919e+00, -2.5842e+00, -1.7926e+00,
         -2.0031e+00, -3.4028e+38],
        [-1.6035e+00, -2.3019e+00, -1.6617e+00, -2.8207e+00, -1.7024e+00,
         -1.7932e+00, -2.2958e+00]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)

In [48]:
a_p[2, :2 + 1]

tensor([0.3252, 0.3354, 0.3395], device='cuda:0', grad_fn=<SliceBackward0>)

In [50]:
l = 0 
for i in range(len(a)):
    print((a_p[i, :i + 1] * (b[i, :i + 1] - a[i, :i + 1])))
    l -= (a_p[i, :i + 1] * (b[i, :i + 1] - a[i, :i + 1])).sum()

tensor([0.], device='cuda:0', grad_fn=<MulBackward0>)
tensor([ 0.2440, -0.5091], device='cuda:0', grad_fn=<MulBackward0>)
tensor([ 0.2374, -0.2493, -0.2435], device='cuda:0', grad_fn=<MulBackward0>)
tensor([ 0.1382, -0.0482, -0.0128, -0.1764], device='cuda:0',
       grad_fn=<MulBackward0>)
tensor([ 0.2147, -0.2276, -0.1847, -0.2223, -0.0009], device='cuda:0',
       grad_fn=<MulBackward0>)
tensor([ 0.1041, -0.0255,  0.0069, -0.1491, -0.0154, -0.0110], device='cuda:0',
       grad_fn=<MulBackward0>)
tensor([ 0.0560, -0.0980,  0.0485, -0.1174,  0.0415,  0.0190, -0.0515],
       device='cuda:0', grad_fn=<MulBackward0>)


In [51]:
l

tensor(1.2323, device='cuda:0', grad_fn=<SubBackward0>)

In [22]:
output.attentions[0].shape

torch.Size([1, 16, 7, 7])

In [12]:
output.attentions[0].shape, output_2.attentions[0].shape

(torch.Size([1, 16, 7, 7]), torch.Size([1, 16, 7, 7]))

In [40]:
mask = torch.ones(1, 16, 7, 7).triu(diagonal=1).to("cuda")

In [45]:
(output.attentions[0] * torch.log((output_2.attentions[0] + mask) / (output.attentions[0] + mask))).sum(axis=3).mean()

tensor(-0.7229, device='cuda:0', grad_fn=<MeanBackward0>)

In [15]:
q = output.attentions[0][0, 0]

In [16]:
p = output_2.attentions[0][0, 0]

In [21]:
(q * torch.log((p) / q)).sum(axis=1)

tensor([    nan,     nan,     nan,     nan,     nan,     nan, -0.6093],
       device='cuda:0', grad_fn=<SumBackward1>)

In [36]:
(q * torch.log((p + upper_tri) / (q + upper_tri))).sum(axis=1).mean()

tensor(-0.2242, device='cuda:0', grad_fn=<MeanBackward0>)

In [33]:
import torch
upper_tri = torch.ones(7, 7).triu()  - torch.eye(7, 7)
upper_tri = upper_tri.to("cuda")

In [37]:
upper_tri = torch.ones(1, 16, 7, 7).triu()  - torch.eye(1, 16, 7, 7)
upper_tri = upper_tri.to("cuda")

TypeError: eye() received an invalid combination of arguments - got (int, int, int, int), but expected one of:
 * (int n, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int n, int m, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [39]:
torch.ones(1, 16, 7, 7).triu(diagonal=1)

tensor([[[[0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1.],
          [0., 0., 0., 0., 1., 1., 1.],
          [0., 0., 0., 0., 0., 1., 1.],
          [0., 0., 0., 0., 0., 0., 1.],
          [0., 0., 0., 0., 0., 0., 0.]],

         [[0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1.],
          [0., 0., 0., 0., 1., 1., 1.],
          [0., 0., 0., 0., 0., 1., 1.],
          [0., 0., 0., 0., 0., 0., 1.],
          [0., 0., 0., 0., 0., 0., 0.]],

         [[0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1.],
          [0., 0., 0., 0., 1., 1., 1.],
          [0., 0., 0., 0., 0., 1., 1.],
          [0., 0., 0., 0., 0., 0., 1.],
          [0., 0., 0., 0., 0., 0., 0.]],

         [[0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1.],
          [0., 0., 0., 0., 1., 1.,